In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,19140
2,Huelva,Confirmados PDIA 14 días,5351
3,Huelva,Tasa PDIA 14 días,"1047,689346696871"
4,Huelva,Confirmados PDIA 7 días,2658
5,Huelva,Total Confirmados,19363
6,Huelva,Curados,10936
7,Huelva,Fallecidos,166


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  19140.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  5147.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 231 personas en los últimos 7 días 

Un positivo PCR cada 115 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,19140.0,2658.0,5351.0,510743.0,520.418293,1047.689347,441.0
Huelva-Costa,11610.0,1700.0,3604.0,288115.0,590.042171,1250.889402,275.0
Huelva (capital),5147.0,622.0,1249.0,143663.0,432.957686,869.395739,113.0
Condado-Campiña,5190.0,709.0,1180.0,155057.0,457.251204,761.010467,111.0
Sierra de Huelva-Andévalo Central,2070.0,226.0,509.0,67571.0,334.463009,753.281733,38.0
Lepe,1530.0,216.0,413.0,27431.0,787.430280,1505.595859,38.0
Ayamonte,836.0,186.0,405.0,20946.0,887.997708,1933.543397,33.0
Isla Cristina,1606.0,281.0,770.0,21264.0,1321.482318,3621.143717,31.0
Punta Umbría,478.0,144.0,207.0,15242.0,944.757906,1358.089490,24.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Villalba del Alcor,373.0,84.0,168.0,3338.0,2516.476932,5032.953865,7.0
Berrocal,14.0,8.0,14.0,312.0,2564.102564,4487.179487,3.0
Encinasola,109.0,4.0,49.0,1305.0,306.513410,3754.789272,0.0
Isla Cristina,1606.0,281.0,770.0,21264.0,1321.482318,3621.143717,31.0
Paymogo,46.0,16.0,34.0,1159.0,1380.500431,2933.563417,1.0
Calañas,144.0,21.0,76.0,2791.0,752.418488,2723.038338,1.0
Villanueva de las Cruces,16.0,1.0,9.0,373.0,268.096515,2412.868633,0.0
San Bartolomé de la Torre,133.0,46.0,76.0,3736.0,1231.263383,2034.261242,9.0
Cerro de Andévalo (El),116.0,15.0,46.0,2364.0,634.517766,1945.854484,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Encinasola,109.0,4.0,49.0,1305.0,306.513410,3754.789272,0.0,0.081633
Villanueva de las Cruces,16.0,1.0,9.0,373.0,268.096515,2412.868633,0.0,0.111111
Minas de Riotinto,95.0,3.0,21.0,3848.0,77.962578,545.738046,1.0,0.142857
Villanueva de los Castillejos,108.0,9.0,39.0,2820.0,319.148936,1382.978723,2.0,0.230769
Almendro (El),32.0,2.0,8.0,826.0,242.130751,968.523002,0.0,0.250000
Aroche,79.0,1.0,4.0,3073.0,32.541490,130.165962,0.0,0.250000
Calañas,144.0,21.0,76.0,2791.0,752.418488,2723.038338,1.0,0.276316
Cerro de Andévalo (El),116.0,15.0,46.0,2364.0,634.517766,1945.854484,0.0,0.326087
Zarza-Perrunal (La),19.0,1.0,3.0,1267.0,78.926598,236.779795,0.0,0.333333
